
## Chapter 4: Variables and Declarations

---

## 4.1 Variable Declaration Keywords

TypeScript, building on JavaScript, provides three keywords for variable declaration: `var`, `let`, and `const`. Understanding the differences between them is crucial for writing predictable, maintainable code.

### 4.1.1 `var` - Legacy Declarations (Why to Avoid)

The `var` keyword is the original way to declare variables in JavaScript. While still valid in TypeScript, it has several problematic behaviors that make it unsuitable for modern development.

**The Problems with `var`:**

```typescript
// Problem 1: Function-scoped, not block-scoped
function varScopeDemo() {
  if (true) {
    var x = 10;
  }
  console.log(x); // 10 - accessible outside the block!
}

// Problem 2: Variable hoisting
console.log(hoisted); // undefined (not an error!)
var hoisted = "I was hoisted";

// This is equivalent to:
var hoisted2; // Declaration moved to top
console.log(hoisted2); // undefined
hoisted2 = "I was hoisted";

// Problem 3: No error on redeclaration
var name = "John";
var name = "Jane"; // No error - silently overwrites
var name = "Bob";  // Still no error
console.log(name); // "Bob"
```

**Hoisting Explained:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    var Hoisting Behavior                             │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Code as written:              │   How JavaScript executes it:     │
│   ──────────────────────────────┼────────────────────────────────── │
│                                 │                                   │
│   console.log(myVar);           │   var myVar; // Hoisted           │
│   // undefined                  │   var myOtherVar; // Hoisted      │
│                                 │                                   │
│   var myVar = "hello";          │   console.log(myVar);             │
│                                 │   // undefined                    │
│   console.log(myOtherVar);      │                                   │
│   // undefined                  │   myVar = "hello";                │
│                                 │                                   │
│   var myOtherVar = "world";     │   console.log(myOtherVar);        │
│                                 │   // undefined                    │
│   console.log(myVar);           │                                   │
│   // "hello"                    │   myOtherVar = "world";           │
│                                 │                                   │
│                                 │   console.log(myVar);             │
│                                 │   // "hello"                      │
│                                                                   │
└─────────────────────────────────────────────────────────────────────┘
```

**Problem: Hoisting Creates Confusion:**

```typescript
// This code runs without error (but is confusing)
function confusingExample() {
  console.log(value); // undefined (not ReferenceError)
  
  if (false) {
    var value = "never assigned";
  }
  
  console.log(value); // still undefined (declaration was hoisted!)
}

// With let, this would be a clear error
function clearExample() {
  // console.log(letValue); // ReferenceError: Cannot access before initialization
  
  if (false) {
    let letValue = "never assigned";
  }
  
  // console.log(letValue); // ReferenceError: letValue is not defined
}
```

**Problem: var in Loops:**

```typescript
// Classic var loop problem
for (var i = 0; i < 3; i++) {
  setTimeout(() => {
    console.log(i);
  }, 100);
}
// Output: 3, 3, 3 (not 0, 1, 2!)

// Why? Because var is function-scoped, and by the time
// the timeouts execute, i has already reached 3

// Solution with let
for (let j = 0; j < 3; j++) {
  setTimeout(() => {
    console.log(j);
  }, 100);
}
// Output: 0, 1, 2 (correct!)

// Each iteration gets its own j variable
```

**Problem: var and Closures:**

```typescript
// Creating functions with var (common bug)
function createCallbacks(): (() => number)[] {
  const callbacks: (() => number)[] = [];
  
  for (var i = 0; i < 3; i++) {
    callbacks.push(() => i);
  }
  
  return callbacks;
}

const callbacks = createCallbacks();
console.log(callbacks[0]()); // 3 (not 0!)
console.log(callbacks[1]()); // 3 (not 1!)
console.log(callbacks[2]()); // 3

// Fix with IIFE (Immediately Invoked Function Expression)
function createCallbacksFixed(): (() => number)[] {
  const callbacks: (() => number)[] = [];
  
  for (var i = 0; i < 3; i++) {
    ((j: number) => {
      callbacks.push(() => j);
    })(i);
  }
  
  return callbacks;
}

// Much simpler with let
function createCallbacksModern(): (() => number)[] {
  const callbacks: (() => number)[] = [];
  
  for (let i = 0; i < 3; i++) {
    callbacks.push(() => i);
  }
  
  return callbacks;
}
```

**Industry Standard: Avoid var**

```typescript
// ❌ Bad: Using var
var userName = "John";
var userAge = 30;

// ✅ Good: Use let for mutable values
let userName = "John";
let userAge = 30;

// ✅ Good: Use const for immutable values
const API_URL = "https://api.example.com";
const MAX_RETRIES = 3;
```

**ESLint Rule to Enforce:**

```json
// .eslintrc.json
{
  "rules": {
    "no-var": "error"
  }
}
```

### 4.1.2 `let` - Block-Scoped Variables

The `let` keyword was introduced in ES6 to address `var`'s shortcomings. It provides block-scoping and prevents many common bugs.

**Block Scoping:**

```typescript
// let is block-scoped
function letScopeDemo() {
  // x is not accessible here
  
  if (true) {
    let x = 10;
    console.log(x); // 10
  }
  
  // console.log(x); // Error: Cannot find name 'x'
}

// Blocks include: if, for, while, switch, try-catch, and standalone blocks
function blockExamples() {
  // Standalone block
  {
    let blockScoped = "only in this block";
    console.log(blockScoped); // OK
  }
  
  // console.log(blockScoped); // Error
  
  // for loop block
  for (let i = 0; i < 3; i++) {
    let loopVar = i * 2;
    console.log(loopVar);
  }
  
  // console.log(i); // Error
  // console.log(loopVar); // Error
  
  // while loop block
  let count = 0;
  while (count < 3) {
    let whileVar = count;
    count++;
    console.log(whileVar);
  }
  
  // console.log(whileVar); // Error
}
```

**No Hoisting Issues:**

```typescript
// Temporal Dead Zone (TDZ)
function temporalDeadZone() {
  // console.log(x); // Error: Cannot access 'x' before initialization
  
  // This is the Temporal Dead Zone for x
  // x exists but cannot be accessed
  
  let x = 10; // x is now initialized
  console.log(x); // 10
}

// The variable is hoisted, but not initialized
// Accessing it before declaration is an error, not undefined
```

**No Redeclaration:**

```typescript
// let prevents accidental redeclaration
let name = "John";
// let name = "Jane"; // Error: Cannot redeclare block-scoped variable 'name'

// This catches bugs early
function processUser(id: number) {
  let user = fetchUser(id);
  
  // Later, if you try to declare again:
  // let user = fetchUser(id + 1); // Error!
  
  // You must use assignment instead:
  user = fetchUser(id + 1); // OK
}

function fetchUser(id: number): string {
  return `User ${id}`;
}
```

**let in Loops (Correct Behavior):**

```typescript
// Each iteration gets its own variable
for (let i = 0; i < 3; i++) {
  setTimeout(() => {
    console.log(i);
  }, 100);
}
// Output: 0, 1, 2

// This works because let creates a new binding for each iteration
// Equivalent to:
for (let i = 0; i < 3; i++) {
  // New i for each iteration
  let iterationI = i;
  setTimeout(() => {
    console.log(iterationI);
  }, 100);
}

// For-of with let
const items = ["a", "b", "c"];
for (let item of items) {
  setTimeout(() => {
    console.log(item);
  }, 100);
}
// Output: "a", "b", "c"

// For-in with let
const obj = { a: 1, b: 2, c: 3 };
for (let key in obj) {
  setTimeout(() => {
    console.log(key, obj[key]);
  }, 100);
}
```

**Type Annotations with let:**

```typescript
// Explicit type annotation
let age: number = 30;
let name: string = "John";
let isActive: boolean = true;

// Type inference
let inferredAge = 30;        // Type: number
let inferredName = "John";   // Type: string

// Declaration without initialization
let userId: number;          // Type: number
let userName: string;        // Type: string

// Must assign before use
// console.log(userId); // Error: Variable 'userId' is used before being assigned

userId = 1001;
console.log(userId); // 1001

// Union types with let
let id: number | string;
id = 123;
id = "abc-123";
```

**Best Practices with let:**

```typescript
// Use let when you need to reassign
function processData(data: string): string {
  let result = data.toLowerCase();
  
  if (result.includes("error")) {
    result = "Error: " + result;
  } else {
    result = "Success: " + result;
  }
  
  return result;
}

// Accumulators
function sumArray(numbers: number[]): number {
  let sum = 0;
  
  for (const num of numbers) {
    sum += num;
  }
  
  return sum;
}

// State that changes
function toggleFeature() {
  let isEnabled = false;
  
  return () => {
    isEnabled = !isEnabled;
    console.log(`Feature is now ${isEnabled ? "enabled" : "disabled"}`);
  };
}

const toggle = toggleFeature();
toggle(); // "Feature is now enabled"
toggle(); // "Feature is now disabled"
```

### 4.1.3 `const` - Constants and Immutability

The `const` keyword declares variables that cannot be reassigned after initialization. It's the preferred choice for values that shouldn't change.

**Basic const Usage:**

```typescript
// const variables cannot be reassigned
const PI = 3.14159;
const API_URL = "https://api.example.com";
const MAX_CONNECTIONS = 10;

// PI = 3.14; // Error: Cannot assign to 'PI' because it is a constant

// Must initialize when declaring
// const UNINITIALIZED; // Error: 'const' declarations must be initialized
```

**const is Block-Scoped:**

```typescript
function constScopeDemo() {
  const x = 10;
  
  if (true) {
    const x = 20; // Different variable, shadowing outer x
    console.log(x); // 20
  }
  
  console.log(x); // 10 (outer x unchanged)
}

// Same scoping rules as let
{
  const blockConst = "only in this block";
  console.log(blockConst); // OK
}
// console.log(blockConst); // Error: Cannot find name 'blockConst'
```

**const with Objects (Important Distinction):**

```typescript
// const prevents reassignment, not mutation
const user = {
  name: "John",
  age: 30
};

// user = { name: "Jane", age: 25 }; // Error: Cannot reassign

// But you CAN mutate the object!
user.name = "Jane"; // OK
user.age = 31;      // OK
console.log(user);  // { name: "Jane", age: 31 }

// Adding properties is allowed
user.email = "jane@example.com";

// To prevent mutation, use Object.freeze
const frozenUser = Object.freeze({
  name: "John",
  age: 30
});

// frozenUser.name = "Jane"; // Error in strict mode (silent fail otherwise)
// Object.freeze is shallow - nested objects can still be mutated
```

**const with Arrays:**

```typescript
const numbers = [1, 2, 3];

// numbers = [4, 5, 6]; // Error: Cannot reassign

// But you CAN mutate the array!
numbers.push(4);       // OK
numbers[0] = 100;      // OK
numbers.pop();         // OK
console.log(numbers);  // [100, 2, 3]

// Use Object.freeze or ReadonlyArray to prevent mutation
const frozenNumbers: readonly number[] = [1, 2, 3];
// frozenNumbers.push(4); // Error: Property 'push' does not exist

// Or use const assertion
const constArray = [1, 2, 3] as const;
// constArray.push(4); // Error
```

**Type Annotations with const:**

```typescript
// Explicit type annotation
const age: number = 30;
const name: string = "John";
const isActive: boolean = true;

// Type inference with const gives literal types
const inferredAge = 30;      // Type: 30 (literal type!)
const inferredName = "John"; // Type: "John" (literal type!)

// To get wider types, use explicit annotation
const widerAge: number = 30; // Type: number
const widerName: string = "John"; // Type: string

// const with union types
type Status = "active" | "inactive";
const userStatus: Status = "active";

// const with complex types
interface Config {
  apiUrl: string;
  timeout: number;
}

const config: Config = {
  apiUrl: "https://api.example.com",
  timeout: 5000
};
```

**When to Use const vs let:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    const vs let Decision Guide                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Question: Will this variable be reassigned?                       │
│                                                                     │
│   ┌─────────────────────────────────────────────────────────────┐  │
│   │                                                             │  │
│   │   NO  ───────────────────▶ Use const                       │  │
│   │   (no reassignment needed)                                  │  │
│   │                                                             │  │
│   │   YES ───────────────────▶ Use let                         │  │
│   │   (reassignment required)                                   │  │
│   │                                                             │  │
│   └─────────────────────────────────────────────────────────────┘  │
│                                                                     │
│   Default to const, use let only when reassignment is necessary    │
│                                                                     │
│   Examples:                                                         │
│   ┌─────────────────────────────────────────────────────────────┐  │
│   │ const PI = 3.14159;           // Never changes              │  │
│   │ const API_URL = "/api";       // Never changes              │  │
│   │ const config = { ... };       // Object reference constant  │  │
│   │                                                             │  │
│   │ let count = 0;               // Will be incremented         │  │
│   │ let result = "";             // Will be modified            │  │
│   │ let isLoading = true;        // Will toggle                 │  │
│   └─────────────────────────────────────────────────────────────┘  │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

**Practical const Patterns:**

```typescript
// Configuration constants
const API_CONFIG = {
  baseUrl: "https://api.example.com",
  version: "v1",
  timeout: 5000
};

// Enum-like constants
const ROLES = {
  ADMIN: "admin",
  USER: "user",
  GUEST: "guest"
} as const;

type Role = typeof ROLES[keyof typeof ROLES]; // "admin" | "user" | "guest"

function hasPermission(role: Role): boolean {
  return role === ROLES.ADMIN;
}

// HTML element references (won't change)
const submitButton = document.getElementById("submit") as HTMLButtonElement;
const emailInput = document.getElementById("email") as HTMLInputElement;

// Regular expressions
const EMAIL_REGEX = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
const PHONE_REGEX = /^\d{10}$/;

// Utility functions (won't be reassigned)
const formatCurrency = (amount: number): string => {
  return `$${amount.toFixed(2)}`;
};

// Event handlers (won't be reassigned)
const handleClick = (event: MouseEvent): void => {
  console.log("Clicked!", event.clientX, event.clientY);
};
```

---

## 4.2 Type Annotations for Variables

Type annotations in TypeScript allow you to explicitly specify the type of a variable. While TypeScript can often infer types, explicit annotations improve code clarity and catch errors early.

### 4.2.1 Basic Syntax

The basic syntax for type annotations uses a colon followed by the type:

```typescript
// Basic syntax: variableName: type = value
let variableName: typeName = value;

// Examples
let age: number = 30;
let name: string = "John Doe";
let isActive: boolean = true;
let nothing: null = null;
let notDefined: undefined = undefined;

// Object types
let user: { name: string; age: number } = {
  name: "Jane",
  age: 25
};

// Array types
let numbers: number[] = [1, 2, 3];
let names: Array<string> = ["Alice", "Bob"];

// Function types
let greet: (name: string) => string;
greet = (name) => `Hello, ${name}!`;
```

**Syntax Variations:**

```typescript
// Declaration with initialization
let age: number = 30;
const PI: number = 3.14159;

// Declaration without initialization (must have type annotation)
let userId: number;
let userName: string;

// Declaration with union type
let id: number | string;
id = 123;
id = "abc-123";

// Declaration with literal type
let status: "active" | "inactive";
status = "active";

// Declaration with any (avoid when possible)
let flexible: any = "can be anything";
flexible = 42;
flexible = { key: "value" };

// Declaration with unknown (safer alternative)
let safe: unknown = "must check before use";
if (typeof safe === "string") {
  console.log(safe.toUpperCase()); // OK
}
```

### 4.2.2 Initializing Variables

Variables can be initialized at declaration or later. The approach differs between `let` and `const`.

**Declaration with Initialization:**

```typescript
// const must be initialized at declaration
const PI = 3.14159;
const API_URL = "https://api.example.com";

// let can be initialized at declaration
let count = 0;
let message = "Hello";

// Or initialized later
let result: string;
let score: number;

// ... later in code
result = "Success";
score = 95;
```

**Definite Assignment Assertion:**

When a variable is initialized elsewhere (e.g., by a framework), use the definite assignment assertion:

```typescript
// Variable will be initialized by framework/test
let userId!: number;
let userName!: string;

// TypeScript trusts that these will be assigned
function initializeUser() {
  userId = 1001;
  userName = "John Doe";
}

// Without the assertion:
let userId2: number;
// console.log(userId2); // Error: Variable 'userId2' is used before being assigned

// With the assertion:
initializeUser();
console.log(userId); // 1001

// Common use case: class properties initialized by dependency injection
class UserService {
  private apiClient!: ApiClient; // Injected by framework
  
  getUser(id: number): User {
    return this.apiClient.fetch(`/users/${id}`);
  }
}
```

**Delayed Initialization:**

```typescript
// Conditional initialization
let discount: number;

if (isPremiumUser) {
  discount = 0.2;
} else {
  discount = 0.1;
}

// Using the nullish coalescing approach
const discount2: number = premiumDiscount ?? 0.1;

// Using ternary
const discount3: number = isPremiumUser ? 0.2 : 0.1;

// Async initialization
let userData: User;

async function loadUser(id: number): Promise<void> {
  userData = await fetchUser(id);
}

// Using promise with then
fetchUser(userId).then((user) => {
  userData = user;
});
```

**Object and Array Initialization:**

```typescript
// Empty object with type annotation
let user: User;
user = { id: 1, name: "John", email: "john@example.com" };

// Empty object with type assertion
let config = {} as Config;
config.apiUrl = "https://api.example.com";

// Partial initialization
interface User {
  id: number;
  name: string;
  email: string;
  role?: string;
}

let newUser: User = {
  id: 1,
  name: "Jane",
  email: "jane@example.com"
  // role is optional
};

// Array initialization
let numbers: number[] = [];
let users: User[] = [];
let matrix: number[][] = [[], [], []];

// Tuple initialization
let tuple: [string, number] = ["age", 30];
let optionalTuple: [string, number?] = ["age"];
```

### 4.2.3 Type Inference in Variable Declarations

TypeScript's type inference often eliminates the need for explicit type annotations.

**Inference with const:**

```typescript
// const infers literal types
const age = 30;        // Type: 30 (not number)
const name = "John";   // Type: "John" (not string)
const isActive = true; // Type: true (not boolean)

// This prevents invalid assignments
const role = "admin";
// role = "user"; // Error: Type '"user"' is not assignable to type '"admin"'

// Array inference
const numbers = [1, 2, 3]; // Type: number[]
const mixed = [1, "two"];  // Type: (string | number)[]

// Object inference
const user = {
  id: 1,
  name: "John",
  email: "john@example.com"
};
// Type: { id: number; name: string; email: string; }
```

**Inference with let:**

```typescript
// let infers broader types
let age = 30;        // Type: number
let name = "John";   // Type: string
let isActive = true; // Type: boolean

// Reassignment is allowed with compatible types
age = 31;           // OK
name = "Jane";      // OK
isActive = false;   // OK

// But type must match
// age = "thirty"; // Error: Type 'string' is not assignable to type 'number'
```

**When Inference is Sufficient:**

```typescript
// Simple values - inference is clear
let count = 0;
const message = "Hello";

// Return values from functions
function getData() {
  return { id: 1, name: "Test" };
}
let data = getData(); // Type inferred from return type

// Array methods
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2); // Type: number[]
const asStrings = numbers.map(String);   // Type: string[]

// Object methods
const obj = { a: 1, b: 2, c: 3 };
const values = Object.values(obj); // Type: number[]
const keys = Object.keys(obj);     // Type: string[]
```

**When to Use Explicit Annotations:**

```typescript
// 1. When type cannot be inferred (no initial value)
let userId: number;
let userName: string;

// 2. When you want a different type than inferred
let specificValue: number | string = 42;

// 3. When the inferred type is too specific
const config = {
  timeout: 5000
}; // Type: { timeout: number }

// If you need to reassign with different values:
let flexibleConfig: { timeout: number } | { retries: number };
flexibleConfig = { timeout: 5000 };
flexibleConfig = { retries: 3 };

// 4. For complex types
interface User {
  id: number;
  name: string;
  email: string;
}

let users: User[] = [];

// 5. For API boundaries
function fetchUser(id: number): User {
  const response = api.get(`/users/${id}`);
  return response as User; // Type assertion at boundary
}
```

**Inference Best Practices:**

```typescript
// ✅ Good: Let inference work for simple cases
let count = 0;
const name = "John";
const isActive = true;

// ✅ Good: Explicit types for complex structures
interface Config {
  apiUrl: string;
  timeout: number;
  retries: number;
}

const config: Config = {
  apiUrl: "https://api.example.com",
  timeout: 5000,
  retries: 3
};

// ✅ Good: Explicit types for function parameters
function processUser(user: User): void {
  console.log(user.name);
}

// ✅ Good: Explicit return types for public APIs
export function calculateTotal(items: CartItem[]): number {
  return items.reduce((sum, item) => sum + item.price, 0);
}

// ❌ Bad: Redundant annotations
let count: number = 0; // Type is obvious
const name: string = "John"; // Type is obvious

// ❌ Bad: Missing types where needed
let data; // Type: any (noImplicitAny error)
data = fetchUser(1);
```

---

## 4.3 Constants and Readonly

TypeScript provides multiple ways to ensure immutability: `const` for variables and `readonly` for properties.

### 4.3.1 `const` vs `readonly`

Understanding the difference between `const` and `readonly` is essential for proper immutability patterns.

**const - Variable Immutability:**

```typescript
// const prevents variable reassignment
const PI = 3.14159;
// PI = 3.14; // Error: Cannot reassign

// But const doesn't make objects immutable
const user = { name: "John", age: 30 };
user.name = "Jane"; // OK - mutation allowed
user.age = 31;      // OK

// Arrays can be mutated
const numbers = [1, 2, 3];
numbers.push(4);    // OK
numbers[0] = 100;   // OK
```

**readonly - Property Immutability:**

```typescript
// readonly prevents property modification
interface User {
  readonly id: number;
  name: string;
  email: string;
}

const user: User = {
  id: 1,
  name: "John",
  email: "john@example.com"
};

user.name = "Jane";    // OK - not readonly
user.email = "jane@example.com"; // OK
// user.id = 2; // Error: Cannot assign to 'id' because it is read-only

// readonly in classes
class Configuration {
  readonly appName: string = "MyApp";
  readonly version: string;
  
  constructor(version: string) {
    this.version = version;
  }
}

const config = new Configuration("1.0.0");
// config.version = "2.0.0"; // Error
```

**Comparison Table:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    const vs readonly Comparison                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Feature              │ const              │ readonly              │
│  ──────────────────────┼────────────────────┼─────────────────────  │
│   What it protects     │ Variable           │ Property              │
│   Reassignment         │ Prevented          │ Prevented             │
│   Mutation             │ Allowed            │ Prevented             │
│   Where used           │ Variable declaration│ Interfaces, classes  │
│   Runtime enforcement  │ JavaScript         │ TypeScript only       │
│   Can be modified      │ No                 │ No (type-level)       │
│                       │                    │                       │
│                                                                   │
└─────────────────────────────────────────────────────────────────────┘
```

**readonly in Arrays and Tuples:**

```typescript
// ReadonlyArray - cannot modify
const numbers: readonly number[] = [1, 2, 3];
// numbers.push(4); // Error: Property 'push' does not exist
// numbers[0] = 100; // Error: Index signature only permits reading

// Alternative syntax
const moreNumbers: ReadonlyArray<number> = [4, 5, 6];

// Readonly tuples
type Point = readonly [number, number];
const origin: Point = [0, 0];
// origin[0] = 1; // Error: Cannot assign

// Create readonly array from regular array
const writable: number[] = [1, 2, 3];
const readonly: readonly number[] = writable;
// readonly.push(4); // Error

// But changes to writable affect readonly!
writable.push(4);
console.log(readonly); // [1, 2, 3, 4]
```

**readonly in Interfaces:**

```typescript
interface User {
  readonly id: number;
  readonly createdAt: Date;
  name: string;
  email: string;
}

interface ReadonlyUser {
  readonly id: number;
  readonly createdAt: Date;
  readonly name: string;
  readonly email: string;
}

// Utility type: Readonly<T>
type ImmutableUser = Readonly<User>;
// All properties become readonly

interface Config {
  readonly api: {
    url: string;
    key: string;
  };
}

const config: Config = {
  api: {
    url: "https://api.example.com",
    key: "secret"
  }
};

// config.api = { ... }; // Error
config.api.url = "changed"; // OK! readonly is shallow
```

**Deep Readonly Pattern:**

```typescript
// Deep readonly utility type
type DeepReadonly<T> = {
  readonly [P in keyof T]: T[P] extends object 
    ? DeepReadonly<T[P]> 
    : T[P];
};

interface NestedConfig {
  api: {
    url: string;
    headers: {
      authorization: string;
      contentType: string;
    };
  };
  database: {
    host: string;
    port: number;
  };
}

const config: DeepReadonly<NestedConfig> = {
  api: {
    url: "https://api.example.com",
    headers: {
      authorization: "Bearer token",
      contentType: "application/json"
    }
  },
  database: {
    host: "localhost",
    port: 5432
  }
};

// config.api.url = "changed"; // Error
// config.api.headers.authorization = "new"; // Error
```

### 4.3.2 Immutable Patterns

Following immutable patterns leads to more predictable and maintainable code.

**Immutable Updates:**

```typescript
// Instead of mutating, create new objects
interface User {
  id: number;
  name: string;
  email: string;
}

const user: User = {
  id: 1,
  name: "John",
  email: "john@example.com"
};

// ❌ Bad: Mutation
// user.name = "Jane";

// ✅ Good: Immutable update (spread operator)
const updatedUser: User = {
  ...user,
  name: "Jane"
};

console.log(user);        // { id: 1, name: "John", ... }
console.log(updatedUser); // { id: 1, name: "Jane", ... }

// Nested updates
interface State {
  user: {
    profile: {
      name: string;
      age: number;
    };
  };
}

const state: State = {
  user: {
    profile: {
      name: "John",
      age: 30
    }
  }
};

// Immutable nested update
const newState: State = {
  ...state,
  user: {
    ...state.user,
    profile: {
      ...state.user.profile,
      age: 31
    }
  }
};
```

**Immutable Array Operations:**

```typescript
const numbers = [1, 2, 3, 4, 5];

// ❌ Bad: Mutation
// numbers.push(6);
// numbers.pop();

// ✅ Good: Immutable operations
const withSix = [...numbers, 6];         // Add element
const withoutLast = numbers.slice(0, -1); // Remove last
const withoutFirst = numbers.slice(1);    // Remove first
const doubled = numbers.map(n => n * 2);  // Transform
const filtered = numbers.filter(n => n > 2); // Filter

// Immutable insert at index
const insertAt = (arr: readonly number[], index: number, value: number) => [
  ...arr.slice(0, index),
  value,
  ...arr.slice(index)
];

const inserted = insertAt(numbers, 2, 100);
// [1, 2, 100, 3, 4, 5]

// Immutable remove at index
const removeAt = (arr: readonly number[], index: number) => [
  ...arr.slice(0, index),
  ...arr.slice(index + 1)
];

const removed = removeAt(numbers, 2);
// [1, 2, 4, 5]
```

**Immutable Object Patterns:**

```typescript
// Freeze objects for runtime immutability
const config = Object.freeze({
  apiUrl: "https://api.example.com",
  timeout: 5000
});

// config.apiUrl = "changed"; // Error in strict mode (silent fail otherwise)

// Deep freeze utility
function deepFreeze<T extends object>(obj: T): Readonly<T> {
  Object.keys(obj).forEach(key => {
    const value = (obj as Record<string, unknown>)[key];
    if (value && typeof value === "object") {
      deepFreeze(value as object);
    }
  });
  return Object.freeze(obj);
}

const deepConfig = deepFreeze({
  api: {
    url: "https://api.example.com",
    headers: {
      auth: "Bearer token"
    }
  }
});

// Both levels are frozen
// deepConfig.api.url = "changed"; // Error
```

**Using Readonly Utility Types:**

```typescript
// Built-in Readonly utility
interface User {
  id: number;
  name: string;
  preferences: {
    theme: string;
    language: string;
  };
}

// Make entire object readonly (shallow)
type ReadonlyUser = Readonly<User>;

// Partial updates
type PartialUser = Partial<User>;

// Required properties
type RequiredUser = Required<User>;

// Pick specific properties
type UserIdAndName = Pick<User, "id" | "name">;

// Omit specific properties
type UserWithoutId = Omit<User, "id">;

// Combine utilities
type ImmutablePartialUser = Readonly<Partial<User>>;
```

**Immutable Class Design:**

```typescript
// Immutable class pattern
class User {
  readonly id: number;
  readonly name: string;
  readonly email: string;
  
  constructor(id: number, name: string, email: string) {
    this.id = id;
    this.name = name;
    this.email = email;
  }
  
  // Methods that return new instances instead of mutating
  withName(newName: string): User {
    return new User(this.id, newName, this.email);
  }
  
  withEmail(newEmail: string): User {
    return new User(this.id, this.name, newEmail);
  }
}

const user = new User(1, "John", "john@example.com");
const updatedUser = user.withName("Jane");

console.log(user.name);        // "John" (unchanged)
console.log(updatedUser.name); // "Jane"

// Using private constructor and factory methods
class ImmutablePoint {
  private constructor(
    readonly x: number,
    readonly y: number
  ) {}
  
  static create(x: number, y: number): ImmutablePoint {
    return new ImmutablePoint(x, y);
  }
  
  static origin(): ImmutablePoint {
    return new ImmutablePoint(0, 0);
  }
  
  translate(dx: number, dy: number): ImmutablePoint {
    return new ImmutablePoint(this.x + dx, this.y + dy);
  }
}

const origin = ImmutablePoint.origin();
const moved = origin.translate(10, 20);
```

---

## 4.4 Destructuring with Types

Destructuring is a convenient way to extract values from arrays or objects. TypeScript provides type support for destructuring patterns.

### 4.4.1 Array Destructuring

Array destructuring allows you to unpack values from arrays into distinct variables.

**Basic Array Destructuring:**

```typescript
// Basic destructuring
const numbers = [1, 2, 3, 4, 5];
const [first, second, third] = numbers;

console.log(first);  // 1
console.log(second); // 2
console.log(third);  // 3

// Skipping elements
const [a, , c] = numbers;
console.log(a); // 1
console.log(c); // 3

// Rest elements
const [head, ...tail] = numbers;
console.log(head); // 1
console.log(tail); // [2, 3, 4, 5]

// Default values
const [x, y, z, w = 0] = [1, 2, 3];
console.log(w); // 0 (default)
```

**Type Annotations in Array Destructuring:**

```typescript
// Type inference works automatically
const [a, b, c] = [1, 2, 3]; // a, b, c are number

// Explicit type annotations
const [first, second]: [number, number] = [1, 2];

// With tuple types
type Point = [number, number];
const point: Point = [10, 20];
const [x, y] = point; // x: number, y: number

// Tuple with different types
type UserTuple = [number, string, boolean];
const userTuple: UserTuple = [1, "John", true];
const [id, name, isActive] = userTuple;
// id: number, name: string, isActive: boolean

// Rest elements in tuples
type StringNumberRest = [string, ...number[]];
const mixed: StringNumberRest = ["hello", 1, 2, 3];
const [str, ...nums] = mixed;
// str: string, nums: number[]
```

**Destructuring in Function Parameters:**

```typescript
// Array destructuring in parameters
function processCoordinates([x, y]: [number, number]): number {
  return Math.sqrt(x * x + y * y);
}

console.log(processCoordinates([3, 4])); // 5

// With rest parameters
function processArray([first, ...rest]: [string, ...number[]]): void {
  console.log(`First: ${first}`);
  console.log(`Rest: ${rest}`);
}

processArray(["numbers", 1, 2, 3]);

// Default array parameter
function processWithDefault([a, b = 0]: [number, number?] = [1]): void {
  console.log(a, b);
}

processWithDefault([2]);    // 2, 0
processWithDefault([2, 3]); // 2, 3
processWithDefault();       // 1, 0
```

**Swapping Variables:**

```typescript
// Swap without temp variable
let a = 1;
let b = 2;

[a, b] = [b, a];

console.log(a); // 2
console.log(b); // 1

// With type annotations
let x: number = 10;
let y: number = 20;

[x, y] = [y, x];

// Works with different types in tuples
let num: number = 1;
let str: string = "hello";
let tuple: [number, string] = [num, str];
[num, str] = [tuple[0], tuple[1]];
```

### 4.4.2 Object Destructuring

Object destructuring extracts properties from objects into variables.

**Basic Object Destructuring:**

```typescript
interface User {
  id: number;
  name: string;
  email: string;
  age: number;
}

const user: User = {
  id: 1,
  name: "John Doe",
  email: "john@example.com",
  age: 30
};

// Basic destructuring
const { id, name, email } = user;

console.log(id);    // 1
console.log(name);  // "John Doe"
console.log(email); // "john@example.com"

// Renaming variables
const { id: userId, name: userName } = user;

console.log(userId);   // 1
console.log(userName); // "John Doe"

// Default values
const { id, name, phone = "N/A" } = user;

console.log(phone); // "N/A"

// Combining renaming and defaults
const { name: fullName = "Anonymous", role = "user" } = user;

console.log(fullName); // "John Doe"
console.log(role);     // "user"
```

**Type Annotations in Object Destructuring:**

```typescript
// Type inference works automatically
const { id, name } = user; // id: number, name: string

// Explicit type annotation on the object
const { x, y }: { x: number; y: number } = { x: 10, y: 20 };

// Using interface for destructured object
interface Point {
  x: number;
  y: number;
}

const point: Point = { x: 100, y: 200 };
const { x, y } = point;

// Renaming with types (type is inferred)
const { x: posX, y: posY } = point;
// posX: number, posY: number

// Type annotation on renamed variable is not needed
// TypeScript automatically infers the type
```

**Nested Object Destructuring:**

```typescript
interface Company {
  name: string;
  address: {
    street: string;
    city: string;
    country: string;
  };
  employees: Array<{
    id: number;
    name: string;
  }>;
}

const company: Company = {
  name: "Tech Corp",
  address: {
    street: "123 Main St",
    city: "San Francisco",
    country: "USA"
  },
  employees: [
    { id: 1, name: "Alice" },
    { id: 2, name: "Bob" }
  ]
};

// Nested destructuring
const {
  name: companyName,
  address: {
    city,
    country
  }
} = company;

console.log(companyName); // "Tech Corp"
console.log(city);        // "San Francisco"
console.log(country);     // "USA"

// Deeply nested with renaming
const {
  address: {
    city: locationCity,
    country: locationCountry
  }
} = company;

// Nested array destructuring
const {
  employees: [firstEmployee, secondEmployee]
} = company;

console.log(firstEmployee.name); // "Alice"

// Nested destructuring with defaults
const config = {
  database: {
    host: "localhost",
    port: 5432
  }
};

const {
  database: {
    host,
    port,
    username = "admin"
  }
} = config;

console.log(username); // "admin" (default)
```

**Rest Properties:**

```typescript
interface User {
  id: number;
  name: string;
  email: string;
  age: number;
  role: string;
}

const user: User = {
  id: 1,
  name: "John",
  email: "john@example.com",
  age: 30,
  role: "admin"
};

// Rest properties
const { id, name, ...rest } = user;

console.log(id);   // 1
console.log(name); // "John"
console.log(rest); // { email: "john@example.com", age: 30, role: "admin" }
// rest type: { email: string; age: number; role: string; }

// Using rest for immutable updates
const updatedUser = {
  ...user,
  ...rest,
  name: "Jane"
};
```

### 4.4.3 Nested Destructuring

Complex nested structures can be destructured with careful type handling.

**Complex Nested Patterns:**

```typescript
interface ApiResponse {
  data: {
    user: {
      profile: {
        id: number;
        name: string;
        settings: {
          theme: string;
          notifications: boolean;
        };
      };
      permissions: string[];
    };
  };
  meta: {
    timestamp: number;
    version: string;
  };
}

const response: ApiResponse = {
  data: {
    user: {
      profile: {
        id: 1,
        name: "John Doe",
        settings: {
          theme: "dark",
          notifications: true
        }
      },
      permissions: ["read", "write"]
    }
  },
  meta: {
    timestamp: Date.now(),
    version: "1.0.0"
  }
};

// Deep nested destructuring
const {
  data: {
    user: {
      profile: {
        name,
        settings: { theme }
      },
      permissions: [firstPermission]
    }
  },
  meta: { version }
} = response;

console.log(name);             // "John Doe"
console.log(theme);            // "dark"
console.log(firstPermission);  // "read"
console.log(version);          // "1.0.0"
```

**Destructuring with Optional Chaining:**

```typescript
interface Config {
  api?: {
    url?: string;
    key?: string;
  };
  features?: {
    darkMode?: boolean;
  };
}

const config: Config = {};

// Safe destructuring with defaults
const {
  api: {
    url: apiUrl = "https://default.api.com",
    key: apiKey = "default-key"
  } = {},
  features: {
    darkMode = false
  } = {}
} = config;

console.log(apiUrl);  // "https://default.api.com"
console.log(apiKey);  // "default-key"
console.log(darkMode); // false
```

**Mixed Array and Object Destructuring:**

```typescript
interface User {
  id: number;
  name: string;
  contacts: Array<{
    type: string;
    value: string;
  }>;
}

const user: User = {
  id: 1,
  name: "John Doe",
  contacts: [
    { type: "email", value: "john@example.com" },
    { type: "phone", value: "555-1234" }
  ]
};

// Mixed destructuring
const {
  name,
  contacts: [
    { value: email },
    { value: phone }
  ]
} = user;

console.log(name);  // "John Doe"
console.log(email); // "john@example.com"
console.log(phone); // "555-1234"
```

### 4.4.4 Default Values in Destructuring

Default values provide fallbacks when destructured properties are undefined.

**Object Default Values:**

```typescript
interface Options {
  width?: number;
  height?: number;
  title?: string;
}

function createChart(options: Options = {}) {
  // Destructuring with defaults
  const {
    width = 800,
    height = 600,
    title = "Untitled Chart"
  } = options;
  
  console.log(`Creating ${width}x${height} chart: ${title}`);
}

createChart(); // Creating 800x600 chart: Untitled Chart
createChart({ width: 1024 }); // Creating 1024x600 chart: Untitled Chart
createChart({ title: "Sales Data" }); // Creating 800x600 chart: Sales Data

// Complete default object
const defaultOptions: Required<Options> = {
  width: 800,
  height: 600,
  title: "Default"
};

function createChartWithOptions(options: Options = {}) {
  const config = { ...defaultOptions, ...options };
  // or
  const { width, height, title } = { ...defaultOptions, ...options };
}
```

**Array Default Values:**

```typescript
// Default values in array destructuring
const [a = 0, b = 0, c = 0] = [1, 2];

console.log(a); // 1
console.log(b); // 2
console.log(c); // 0 (default)

// Default with undefined
const [x = "default", y = "default"] = [undefined, "value"];

console.log(x); // "default"
console.log(y); // "value"

// Function returning array with defaults
function getCoordinates(): [number?, number?] {
  return [];
}

const [lat = 0, lng = 0] = getCoordinates();
console.log(lat, lng); // 0, 0
```

**Complex Default Patterns:**

```typescript
interface User {
  id: number;
  name: string;
  preferences?: {
    theme?: string;
    language?: string;
    notifications?: {
      email?: boolean;
      push?: boolean;
    };
  };
}

const defaultPreferences = {
  theme: "light",
  language: "en",
  notifications: {
    email: true,
    push: false
  }
};

function setupUser(user: User) {
  // Nested defaults with spread
  const {
    preferences = {}
  } = user;
  
  const preferencesWithDefaults = {
    ...defaultPreferences,
    ...preferences,
    notifications: {
      ...defaultPreferences.notifications,
      ...preferences?.notifications
    }
  };
  
  // Or use destructuring with defaults at each level
  const {
    preferences: {
      theme = "light",
      language = "en",
      notifications: {
        email = true,
        push = false
      } = {}
    } = {}
  } = user;
  
  return { theme, language, email, push };
}
```

**Function Parameter Defaults:**

```typescript
// Function with destructured defaults
interface Config {
  apiUrl?: string;
  timeout?: number;
  retries?: number;
}

function fetchData(
  endpoint: string,
  {
    apiUrl = "https://api.default.com",
    timeout = 5000,
    retries = 3
  }: Config = {}
) {
  console.log(`Fetching ${apiUrl}${endpoint}`);
  console.log(`Timeout: ${timeout}ms, Retries: ${retries}`);
}

fetchData("/users");
// Fetching https://api.default.com/users
// Timeout: 5000ms, Retries: 3

fetchData("/users", { timeout: 10000 });
// Fetching https://api.default.com/users
// Timeout: 10000ms, Retries: 3

// Required properties with optional
interface UserOptions {
  name: string;
  age: number;
  email?: string;
}

function createUser({
  name,
  age,
  email = "no-email"
}: UserOptions) {
  console.log({ name, age, email });
}

createUser({ name: "John", age: 30 });
// { name: "John", age: 30, email: "no-email" }
```

---

## 4.5 Variable Scoping and Hoisting

Understanding scoping rules and hoisting behavior is essential for writing predictable TypeScript code.

### 4.5.1 Block Scope

Block scope means variables are only accessible within the block where they're defined.

**Block Scope with let and const:**

```typescript
function blockScopeDemo() {
  // Variables declared here are function-scoped
  
  if (true) {
    // Block-scoped variables
    let blockLet = "let variable";
    const blockConst = "const variable";
    
    console.log(blockLet);   // OK
    console.log(blockConst); // OK
  }
  
  // console.log(blockLet);   // Error: Cannot find name 'blockLet'
  // console.log(blockConst); // Error: Cannot find name 'blockConst'
}

// All blocks create scope
{
  let inBlock = "inside";
  console.log(inBlock); // OK
}

// console.log(inBlock); // Error

// Loop blocks
for (let i = 0; i < 3; i++) {
  let loopVar = `iteration ${i}`;
  console.log(loopVar);
}

// console.log(i);       // Error
// console.log(loopVar); // Error

// Try-catch blocks
try {
  let tryVar = "in try";
  throw new Error("test");
} catch (error) {
  let catchVar = "in catch";
  console.log(error); // OK
  // console.log(tryVar); // Error
} finally {
  let finallyVar = "in finally";
  // console.log(catchVar); // Error
}
```

**Variable Shadowing:**

```typescript
// Shadowing - inner variable hides outer
let x = "outer";

function shadowingDemo() {
  let x = "inner"; // Shadows outer x
  console.log(x); // "inner"
  
  if (true) {
    let x = "innermost"; // Shadows inner x
    console.log(x); // "innermost"
  }
  
  console.log(x); // "inner"
}

shadowingDemo();
console.log(x); // "outer"

// Avoid shadowing for clarity
let value = "global";

function goodPractice() {
  let localValue = "local"; // Different name
  console.log(localValue);
  console.log(value); // Can still access global
}
```

**Block Scope in Switch Statements:**

```typescript
// Switch statements need blocks for each case
function getStatusMessage(status: number): string {
  let message: string;
  
  switch (status) {
    case 200: {
      const code = "OK";
      message = code;
      break;
    }
    case 404: {
      const code = "Not Found"; // Different scope, no conflict
      message = code;
      break;
    }
    default: {
      const code = "Unknown";
      message = code;
    }
  }
  
  return message;
}

// Without blocks, there would be redeclaration errors
```

### 4.5.2 Function Scope

Function scope means variables are accessible throughout the function where they're declared.

**Function Scope with var:**

```typescript
function functionScopeDemo() {
  // var is function-scoped
  if (true) {
    var functionScoped = "accessible everywhere in function";
  }
  
  console.log(functionScoped); // OK - var hoists to function level
  
  for (var i = 0; i < 3; i++) {
    var loopVar = i;
  }
  
  console.log(i);        // 3 - still accessible
  console.log(loopVar);  // 2 - still accessible
}

// This is why var is problematic
function varProblemDemo() {
  // All var declarations are hoisted to the top
  console.log(declaredLater); // undefined (not error!)
  
  if (false) {
    var declaredLater = "never assigned";
  }
  
  console.log(declaredLater); // undefined (declaration hoisted)
}
```

**Function Parameters:**

```typescript
// Parameters are function-scoped
function paramScope(a: number, b: number): void {
  // a and b are accessible throughout the function
  console.log(a, b);
  
  if (true) {
    console.log(a, b); // Still accessible
    let c = a + b;     // Block-scoped
  }
  
  // console.log(c); // Error
}

// Default parameters
function defaultParams(
  a: number,
  b: number = a * 2 // Can reference earlier parameters
): void {
  console.log(a, b);
}

defaultParams(5); // 5, 10
```

**Closures and Scope:**

```typescript
// Closures capture variables from outer scope
function createCounter(): () => number {
  let count = 0; // Function-scoped to createCounter
  
  return () => {
    count++; // Closure captures count
    return count;
  };
}

const counter = createCounter();
console.log(counter()); // 1
console.log(counter()); // 2
console.log(counter()); // 3

// Each call creates a new scope
const counter2 = createCounter();
console.log(counter2()); // 1 (separate count)

// Closure with block scope
function createMultipliers(): Array<(x: number) => number> {
  const multipliers: Array<(x: number) => number> = [];
  
  for (let i = 1; i <= 3; i++) {
    // Each iteration has its own i
    multipliers.push((x: number) => x * i);
  }
  
  return multipliers;
}

const [double, triple, quadruple] = createMultipliers();
console.log(double(5));   // 10
console.log(triple(5));   // 15
console.log(quadruple(5)); // 20
```

### 4.5.3 Global Scope

The global scope is the outermost scope. Variables declared globally are accessible everywhere.

**Global Scope in Node.js:**

```typescript
// In Node.js, global scope is module-scoped by default
// Variables declared at the top level are module-scoped, not truly global

// Module-scoped variable
let moduleVar = "module level";

// To access truly global scope in Node.js
declare global {
  var globalVar: string;
}

globalThis.globalVar = "truly global";

// Or use global object
// global.globalVar = "truly global"; // Node.js specific
```

**Global Scope in Browser:**

```typescript
// In browsers, var creates global variables on window
// var globalInBrowser = "on window object";

// let and const don't attach to window
let notOnWindow = "not global";

// Accessing global variables
declare const window: Window & {
  myGlobalVar?: string;
};

window.myGlobalVar = "global value";
```

**Avoiding Global Pollution:**

```typescript
// ❌ Bad: Global variables
let userName = "John";
let userAge = 30;

function processUser() {
  console.log(userName, userAge);
}

// ✅ Good: Encapsulate in modules or objects
namespace UserModule {
  const userName = "John";
  const userAge = 30;
  
  export function processUser() {
    console.log(userName, userAge);
  }
}

UserModule.processUser();

// ✅ Better: Use ES modules
// user.ts
export const userName = "John";
export const userAge = 30;
export function processUser() {
  console.log(userName, userAge);
}

// main.ts
import { processUser } from "./user";
processUser();
```

### 4.5.4 Temporal Dead Zone

The Temporal Dead Zone (TDZ) is the period between entering a scope and the variable being declared.

**Understanding TDZ:**

```typescript
function tdzDemo() {
  // TDZ starts here for myVar
  
  // console.log(myVar); // Error: Cannot access 'myVar' before initialization
  
  // Still in TDZ
  const myFunc = () => myVar; // OK to capture in closure
  
  // Still in TDZ
  
  let myVar = "initialized"; // TDZ ends here
  
  console.log(myVar); // "initialized"
  console.log(myFunc()); // "initialized"
}

// TDZ applies to let and const, not var
function varNoTDZ() {
  console.log(varVar); // undefined (hoisted, no TDZ)
  var varVar = "value";
}
```

**TDZ with const:**

```typescript
function constTDZ() {
  // console.log(VALUE); // Error: TDZ
  
  const VALUE = 42;
  console.log(VALUE); // 42
}

// TDZ prevents accessing before initialization
// This catches bugs where you might accidentally use undefined values
```

**TDZ in Different Scopes:**

```typescript
// TDZ in block scope
function blockTDZ() {
  let x = "outer";
  
  if (true) {
    // TDZ for x starts here
    // console.log(x); // Error: Cannot access 'x' before initialization
    
    let x = "inner"; // New x, TDZ ends
    console.log(x); // "inner"
  }
  
  console.log(x); // "outer"
}

// TDZ in for loops
function loopTDZ() {
  for (let i = 0; i < 3; i++) {
    // Each iteration has its own TDZ for i
    console.log(i);
  }
}

// TDZ with default parameters
function defaultParamTDZ(a: number = b, b: number = 2) {
  // Error: Cannot access 'b' before initialization
  // TDZ prevents referencing later parameters
}

function correctParams(a: number, b: number = a) {
  // OK - a is already initialized
}
```

**TDZ Visualization:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Temporal Dead Zone Visualization                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   function example() {                                              │
│                                                                     │
│   ┌────────────────────────────────────────────────────────────┐   │
│   │                    TDZ for 'value'                          │   │
│   │                                                            │   │
│   │   // console.log(value);  // ❌ ReferenceError             │   │
│   │   // if (value) { }       // ❌ ReferenceError             │   │
│   │   // const x = value;     // ❌ ReferenceError             │   │
│   │                                                            │   │
│   │   const capture = () => value; // ✅ OK (doesn't evaluate) │   │
│   │                                                            │   │
│   └────────────────────────────────────────────────────────────┘   │
│                                                                     │
│   let value = "initialized"; // ← TDZ ends here                    │
│                                                                     │
│   console.log(value); // ✅ OK: "initialized"                      │
│   console.log(capture()); // ✅ OK: "initialized"                  │
│                                                                     │
│   }                                                                 │
│                                                                     │
│   Why TDZ?                                                         │
│   • Catches errors before they cause subtle bugs                   │
│   • Forces intentional declaration before use                       │
│   • Prevents accessing uninitialized state                          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

**Practical TDZ Examples:**

```typescript
// Common mistake: referencing before declaration
class Example {
  private data = this.processData(this.rawData); // Error if rawData not initialized
  private rawData = "some data";
  
  private processData(data: string): string {
    return data.toUpperCase();
  }
}

// Fix: reorder declarations
class FixedExample {
  private rawData = "some data";
  private data = this.processData(this.rawData); // OK
  
  private processData(data: string): string {
    return data.toUpperCase();
  }
}

// TDZ with inheritance
class Parent {
  protected value = 10;
}

class Child extends Parent {
  // private calculated = this.value * 2; // Error if before super()
  
  private calculated: number;
  
  constructor() {
    super();
    this.calculated = this.value * 2; // OK
  }
}
```

---

## 4.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we covered variables and declarations in depth:

**Key Takeaways:**

1. **Variable Declaration Keywords**:
   - `var` - Legacy, function-scoped, hoisted (avoid in modern code)
   - `let` - Block-scoped, no hoisting issues (use for mutable values)
   - `const` - Block-scoped, cannot reassign (use for immutable references)

2. **Type Annotations**:
   - Basic syntax: `let name: type = value`
   - Let TypeScript infer when type is obvious
   - Always annotate function parameters
   - Use explicit types when inference isn't sufficient

3. **Constants and Readonly**:
   - `const` prevents variable reassignment
   - `readonly` prevents property modification
   - Use immutable patterns for safer code
   - Deep readonly for nested objects

4. **Destructuring**:
   - Array destructuring with `[a, b] = array`
   - Object destructuring with `{ x, y } = object`
   - Nested destructuring for complex structures
   - Default values for optional properties

5. **Scoping**:
   - Block scope: `let` and `const`
   - Function scope: `var`
   - Global scope: avoid pollution
   - Temporal Dead Zone prevents early access

### Practical Exercises

**Exercise 1: Variable Declarations**

Refactor the following code to use modern variable declarations:

```typescript
// Refactor this code - replace var with let/const
var config = {
  apiUrl: "https://api.example.com",
  timeout: 5000
};

var users = [];
users.push({ id: 1, name: "John" });

for (var i = 0; i < 3; i++) {
  setTimeout(function() {
    console.log(i);
  }, 100);
}
```

**Exercise 2: Type Annotations**

Add appropriate type annotations:

```typescript
// Add type annotations where needed

let userId; // Should be number
let userName; // Should be string
let isActive; // Should be boolean

function processUser(user) { // Add types for user
  return user.name.toUpperCase();
}

interface Config {
  // Define the interface
}

let config: Config = {
  apiUrl: "https://api.example.com",
  retries: 3,
  timeout: 5000
};
```

**Exercise 3: Destructuring**

Use destructuring to extract the following values:

```typescript
const response = {
  data: {
    users: [
      { id: 1, name: "John", email: "john@example.com" },
      { id: 2, name: "Jane", email: "jane@example.com" }
    ],
    total: 2
  },
  status: 200,
  message: "Success"
};

// Extract: first user's name, total, and status using destructuring
```

**Exercise 4: Immutable Patterns**

Implement immutable update patterns:

```typescript
interface User {
  id: number;
  name: string;
  email: string;
  preferences: {
    theme: string;
    notifications: boolean;
  };
}

const user: User = {
  id: 1,
  name: "John",
  email: "john@example.com",
  preferences: {
    theme: "light",
    notifications: true
  }
};

// Create updated versions without mutation:
// 1. Update name to "Jane"
// 2. Update theme to "dark"
// 3. Update both name and email
```

**Exercise 5: Scope and TDZ**

Identify and fix the errors:

```typescript
function scopeExercise() {
  console.log(x); // What happens?
  console.log(y); // What happens?
  
  var x = 10;
  let y = 20;
  
  if (true) {
    console.log(x); // What happens?
    console.log(y); // What happens?
    
    var x = 30;
    let y = 40;
  }
  
  console.log(x); // What value?
  console.log(y); // What value?
}
```

### Additional Resources

- **TypeScript Handbook - Variable Declarations**: https://www.typescriptlang.org/docs/handbook/variable-declarations.html
- **MDN - let**: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Statements/let
- **MDN - const**: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Statements/const
- **MDN - Destructuring**: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Destructuring_assignment

---

## Coming Up Next: Chapter 5 - Arrays and Tuples

In the next chapter, we will explore arrays and tuples in depth:

- Arrays in TypeScript with type annotations
- Array methods and type safety
- Readonly arrays
- Understanding and using tuples
- Advanced tuple patterns

Understanding arrays and tuples is essential for working with collections of data in TypeScript.

